In [2]:
import os
raiz = os.getcwd().replace('\\notebooks', '')
os.chdir(raiz)

import pandas as pd
import pickle
from sklearn.ensemble import RandomForestRegressor

from utils import create_predictors
from skforecast.model_selection import grid_search_forecaster
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom

import requests
import pandas as pd
from datetime import datetime


In [3]:
# Datos 
steps = 36
n_datos_entrenar = 200
path_fichero = 'bitcoin.csv'
path_modelo = 'model.pickle'



In [4]:
# Extraigo la info de Bitcoin
url = 'https://api.blockchain.info/charts/transactions-per-second?timespan=all&sampled=false&metadata=false&cors=true&format=json'
resp = requests.get(url)

bitcoin = pd.DataFrame(resp.json()['values'])



In [5]:
# Concierto las fecha UNIX en timestamp
bitcoin['x'] = [datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S') for x in bitcoin['x']]
bitcoin['x'] = pd.to_datetime(bitcoin['x'])

# Doy nombre a las columnas
bitcoin.columns = ['date', 'transactions']

# Guardo el fichero en csv
bitcoin.to_csv(path_fichero, index = False)



In [6]:
# Leo los datos
datos = pd.read_csv(path_fichero)

# Fijo el tipo de datos
datos['date'] = pd.to_datetime(datos['date'])
datos['date'] = datos['date'].dt.round('H')

datos_agrupados = datos\
  .groupby('date')\
  .sum()\
  .reset_index()

datos_agrupados

,date,transactions
0,2016-06-11 00:00:00,5.850000
1,2016-06-11 01:00:00,6.050000
2,2016-06-11 02:00:00,9.050000
3,2016-06-11 03:00:00,6.050000
4,2016-06-11 04:00:00,12.650000
...,...,...
47958,2021-12-01 07:00:00,9.350000
47959,2021-12-01 08:00:00,16.533333
47960,2021-12-01 09:00:00,10.566667
47961,2021-12-01 10:00:00,17.150000


In [ ]:
# Pongo bien los datos
datos = datos.set_index('date')
datos = datos['transactions']

# Split entre train y test
datos_train = datos[ -n_datos_entrenar:-steps]
datos_test  = datos[-steps:]



In [ ]:
# Grid search de hiperparámetros
forecaster_rf = ForecasterAutoregCustom(
                    regressor      = RandomForestRegressor(random_state=123),
                    fun_predictors = create_predictors,
                    window_size    = 20
                )

# Hiperparámetros del regresor
param_grid = {'n_estimators': [100, 500],
              'max_depth': [3, 5, 10]}

resultados_grid = grid_search_forecaster(
                        forecaster  = forecaster_rf,
                        y           = datos_train,
                        param_grid  = param_grid,
                        steps       = 10,
                        method      = 'cv',
                        metric      = 'mean_squared_error',
                        initial_train_size    = int(len(datos_train)*0.5),
                        allow_incomplete_fold = True,
                        return_best = True,
                        verbose     = False
                    )



In [ ]:
forecaster_rf.predict(datos_test)

In [ ]:
# Guardo el modelo
ultima_fecha_entrenamiento = datos_test.index[-1].strftime('%Y-%m-%d %H:%M:%S')
pickle.dump(ultima_fecha_entrenamiento, open('ultima_fecha_entrenamiento.pickle', 'wb'))
pickle.dump(forecaster_rf, open(path_modelo, 'wb'))